In [4]:
import cv2
import mediapipe as mp
import numpy as np
import time
import itertools


ImportError: cannot import name 'builder' from 'google.protobuf.internal' (c:\Users\ahmed\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\internal\__init__.py)

In [ ]:
def calculate_ear(eye_landmarks):
    # Extract specific eye landmark points
    p2 = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    p6 = np.array([eye_landmarks[5].x, eye_landmarks[5].y])
    p3 = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    p5 = np.array([eye_landmarks[4].x, eye_landmarks[4].y])
    p1 = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    p4 = np.array([eye_landmarks[3].x, eye_landmarks[3].y])
    
    # Calculate the distance between the landmarks
    ear = (np.linalg.norm(p2 - p6) + np.linalg.norm(p3 - p5)) / (2 * np.linalg.norm(p1 - p4))
    
    return ear


In [ ]:
def compute_AVG_EAR(left_ear,right_ear):
    return round(((right_ear+left_ear)/2),3)


In [ ]:
import pygame

def play_alert_sound():
    pygame.mixer.init()
    pygame.mixer.music.load("alert.mp3")  # Replace "alert_sound.wav" with the path to your sound file.
    pygame.mixer.music.play()

pygame 2.5.0 (SDL 2.28.0, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.7)

mp_drawing = mp.solutions.drawing_utils

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)


cap = cv2.VideoCapture(0)
consecutive_frames=0
left_consecutive=0
right_consecutive=0
up_consecutive=0
down_consecutive=0
is_sound_enabled = True
while cap.isOpened():
    success, image = cap.read()

    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # To improve performance
    image.flags.writeable = False
    
    # Get the result
    results = face_mesh.process(image)
    
    # To improve performance
    image.flags.writeable = True
    
    # Convert the color space from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)

                    # Get the 2D Coordinates
                    face_2d.append([x, y])

                    # Get the 3D Coordinates
                    face_3d.append([x, y, lm.z])       
            
            # Convert it to the NumPy array
            face_2d = np.array(face_2d, dtype=np.float64)

            # Convert it to the NumPy array
            face_3d = np.array(face_3d, dtype=np.float64)
            
            #CAM CALIBRATION

            # The camera matrix
            focal_length = 1 * img_w

            cam_matrix = np.array([ [focal_length, 0, img_h / 2],
                                    [0, focal_length, img_w / 2],
                                    [0, 0, 1]])

            # The distortion parameters
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Solve PnP
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            # Get rotational matrix
            rmat, jac = cv2.Rodrigues(rot_vec)

            # Get angles
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

            # Get the y rotation degree
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
        
            # See where the user's head tilting
            if y < -17:
                text = "Looking Left"
                left_consecutive+=1
            elif y > 15:
                text = "Looking Right"
                right_consecutive+=1
            elif x < -10:
                text = "Looking Down"
                down_consecutive+=1
            elif x > 23:
                text = "Looking Up"
                up_consecutive+=1
            else:
                text = "Forward"
                left_consecutive=0
                right_consecutive=0
                up_consecutive=0
                down_consecutive=0


            # Display the nose direction
            nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)

            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_2d[0] + y * 10) , int(nose_2d[1] - x * 10))
            
            cv2.line(image, p1, p2, (255, 0, 0), 3)

            # Add the text on the image
            cv2.putText(image, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
            cv2.putText(image, "x: " + str(np.round(x,2)), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, "y: " + str(np.round(y,2)), (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, "z: " + str(np.round(z,2)), (500, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            #ADDING EAR
            LEFT_EYE_INDEXES = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYE)))
            RIGHT_EYE_INDEXES = list(set(itertools.chain(*mp_face_mesh.FACEMESH_RIGHT_EYE)))

            # Draw the left eye landmarks
            
            """
            left_ear = calculate_ear(LEFT_EYE_INDEXES)
            print(left_ear)
            right_ear=calculate_ear(RIGHT_EYE_INDEXES)
            print('right=',right_ear)
            EAR=compute_AVG_EAR(left_ear,right_ear)
            cv2.putText(image,f"EAR={EAR}", (500, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)"""
            # Right eye landmarks indices
            RIGHT_EYE_INDEXE = [362, 385, 387, 263, 373, 380]

            # left eye landmarks indices
            LEFT_EYE_INDEXE = [33, 160, 158, 133, 153, 144]

            # Function to calculate EAR

            # Calculate the EAR for the left eye
            left_eye_landmarks = [results.multi_face_landmarks[0].landmark[idx] for idx in LEFT_EYE_INDEXE]
            left_eye_ear = calculate_ear(left_eye_landmarks)

            # Calculate the EAR for the right eye
            right_eye_landmarks = [results.multi_face_landmarks[0].landmark[idx] for idx in RIGHT_EYE_INDEXE]
            right_eye_ear = calculate_ear(right_eye_landmarks)

            average_ear = round(((left_eye_ear + right_eye_ear) / 2),3)
            cv2.putText(image,f"EAR={average_ear}", (500, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec)

            for idx in LEFT_EYE_INDEXES:
                landmark_pt = face_landmarks.landmark[idx]
                x, y = int(landmark_pt.x * image.shape[1]), int(landmark_pt.y * image.shape[0])
                cv2.circle(image, (x, y), 1, (0, 255, 0), 1)

            # Draw the right eye landmarks
            for idx in RIGHT_EYE_INDEXES:
                landmark_pt = face_landmarks.landmark[idx]
                x, y = int(landmark_pt.x * image.shape[1]), int(landmark_pt.y * image.shape[0])
                cv2.circle(image, (x, y), 1, (0, 255, 0), 1)
            if average_ear < 0.25:
                    consecutive_frames += 1
                    sound_played = False
            else:
                consecutive_frames = 0
            condition=(consecutive_frames > 20 or up_consecutive>50 or down_consecutive>50 or right_consecutive>50 or left_consecutive>50)
            if condition and not sound_played:
            # Display the alert overlay on the frame
            
                cv2.putText(image, "ALERT!", (image.shape[0]//2, image.shape[1]//2), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5, cv2.LINE_AA)
                if is_sound_enabled:
                    play_alert_sound()
                    sound_played = True
                consecutive_frames = 0
                up_consecutive=0
                down_consecutive=0
                right_consecutive=0
                left_consecutive=0

            if (average_ear > 0.25 or not condition):
                sound_played = False



    #cv2.putText(image, f'FPS: {int(fps)}', (20,450), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
        
       
    cv2.imshow('Head Pose Estimation', image)

    if cv2.waitKey(1) == ord('e'):
        break


cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import dlib
import numpy as np
import itertools
import pygame
# Load the face detector and landmark predictor from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Initialize variables for consecutive frame tracking
consecutive_frames = 0
left_consecutive = 0
right_consecutive = 0
up_consecutive = 0
down_consecutive = 0
is_sound_enabled = True

# Load an alert sound function (you need to implement this)
def play_alert_sound():
    pygame.mixer.init()
    pygame.mixer.music.load("alert.mp3")  # Replace "alert_sound.wav" with the path to your sound file.
    pygame.mixer.music.play()

# Function to calculate EAR for a set of landmarks
def calculate_ear(eye_landmarks):
    # Extract specific eye landmark points
    p2 = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    p6 = np.array([eye_landmarks[5].x, eye_landmarks[5].y])
    p3 = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    p5 = np.array([eye_landmarks[4].x, eye_landmarks[4].y])
    p1 = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    p4 = np.array([eye_landmarks[3].x, eye_landmarks[3].y])
    
    # Calculate the distance between the landmarks
    ear = (np.linalg.norm(p2 - p6) + np.linalg.norm(p3 - p5)) / (2 * np.linalg.norm(p1 - p4))
    
    return ear

# Main loop
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()

    # Convert the color space from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect faces in the image
    faces = detector(image)

    for face in faces:
        # Get facial landmarks for the detected face
        landmarks = predictor(image, face)

        # Convert the landmarks to NumPy array for ease of use
        landmarks = np.array([[point.x, point.y] for point in landmarks.parts()])

        # Perform pose estimation and EAR calculation using the landmarks
        # You can implement this logic based on your original code

        # Display your results on the image as needed

    cv2.imshow('Head Pose Estimation', image)

    if cv2.waitKey(1) == ord('e'):
        break

cap.release()
cv2.destroyAllWindows()
